In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import gzip as gz
import ast
import re
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
info=[]
for i in gz.open(r'C:\Users\Usuario\Desktop\HENRY\gonzalo1\users_items.json.gz'):
     info.append(ast.literal_eval(i.decode('utf-8')))

df_user_items = pd.DataFrame(info)

In [3]:
# Con las funciones explode y normalize desanidamos las columna "items" para poder
# disponibilizar esos datos para consumirlos en etapas posteriores
df_items_exploded = df_user_items.explode(['items'])
df_items_desanidado = pd.json_normalize(df_items_exploded['items'])

# Por otra parte reindexamos los DF
df_items_exploded.reset_index(inplace=True)
df_items_desanidado.reset_index(inplace=True)

# Concatenamos y eliminamos la columna que previamente desanidamos
user_items = pd.concat([df_items_exploded,df_items_desanidado], axis=1)
user_items = user_items.drop(columns = ['items'])
user_items

,index,user_id,items_count,steam_id,user_url,index,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,20,Team Fortress Classic,0.0,0.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,30,Day of Defeat,7.0,0.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,40,Deathmatch Classic,0.0,0.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5170010,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170010,373330,All Is Dust,0.0,0.0
5170011,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170013,519140,Minds Eyes,3.0,3.0


In [4]:
user_items['item_id']=user_items['item_id'].astype(float)

In [6]:
user_items = user_items.copy()

In [9]:
archivocsv = 'df/user_itemsfinal.csv'
user_items.to_csv(archivocsv, index=False, encoding='utf-8')

In [11]:
#Transformo el archivo csv a parquet
#Leo el archivo csv
user_items= pd.read_csv("df/user_itemsfinal.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "df/user_items.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(user_items)
pq.write_table(table,output_file)